In [1]:
import sys
sys.path.append('../src/')
import os
import copy


import numpy as np
import sklearn
from sklearn import metrics
import tensorly as tn
import math
# import matlab
import matlab.engine
import matplotlib.pyplot as plt
import time

# import milk_tools

### Load Dataset

In [8]:
# spectra_path = '../data/casein_data.npy' # dummy dataset name for now, 999 wavenumber & 1 target
# spectra_path = '../data/2024_caseinval_data.npy' # 2024 company test data
# spectra_path = '../data/2022_caseinval_data.npy' # 2022 company test data
# spectra_path = '../data/2024_caseinvaldata_reduced.npy' # 2024 company test data (reduced)
# spectra_path = '../data/2022_caseinvaldata_reduced.npy' # 2022 company test data (reduced)
spectra_path = '../data/caseintraindata_reduced.npy'    # training data (reduced)
raw_spectra_and_labels = np.load(spectra_path)
print(f"Targets shape: {raw_spectra_and_labels.shape}")

# tensorlab_path = '~/ware/matlab_tools/tensorlab'
tensorlab_path = '~/tools/matlab_tools/Tensorlab'

eng = matlab.engine.start_matlab()
s = eng.genpath(tensorlab_path)
eng.addpath(s,nargout=0)

num_spectra = raw_spectra_and_labels.shape[0]
num_waveNums = raw_spectra_and_labels.shape[1] - 1

spectra = copy.deepcopy(raw_spectra_and_labels[:,:num_waveNums])
print(spectra.shape)
spectra = np.transpose(spectra)
print(spectra.shape)
labels = copy.deepcopy(raw_spectra_and_labels[:,-1])
print(labels.shape)

num_splits = num_spectra // 100
last_split = num_spectra % 100

lwn_spectra = []
for i in range(num_splits):
    lo = i * 100
    hi = i * 100 + 100
    L = eng.loewnerize(np.array(spectra[:,lo:hi]), nargout=1)
    lwn_spectra.append(np.array(L))
L = eng.loewnerize(np.array(spectra[:,100*num_splits:]), nargout=1)
lwn_spectra.append(np.array(L))
# for i in range(num_spectra):
#     tmp_spec = np.array(spectra[i,:])
#     print(tmp_spec.shape)
    # tmp_l = eng.loewnerize(tmp_spec,'dim',1,'t',np.linspace(0,1,num_waveNums), nargout=1)
print(len(lwn_spectra))




Targets shape: (3282, 53)
(3282, 52)
(52, 3282)
(3282,)
33


In [9]:
lwn_spectra = np.dstack(lwn_spectra)
print(lwn_spectra.shape)

(26, 26, 3282)


In [9]:
lwn_path = '/Users/matthewmerris/dev/research/milk/results/lwn_spec.npz'
np.savez(lwn_path, lwn_spectra, labels)


In [10]:
lwn_path = '/Users/matthewmerris/dev/research/milk/results/lwn_spec_train_reduced.npz'
np.savez(lwn_path, lwn_spectra, labels)

In [37]:
print(L['dim'])
# print(L['ind'])
print(L['order'])


1.0
2.0


In [ ]:
# Save lwn_spectra (500 x 499 x 3282) and labels (

In [16]:
eng.quit()
